# Data Preparation

This notebook prepares the raw dataset for exploratory data analysis and machine learning models.

##1. Dependencies

In [1]:
# Packages
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.special import gammaln
from scipy.stats import poisson
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive")

# Importing data and quick inspection
df = pd.read_csv("/content/drive/My Drive/Goal_Prediction_Model/data/league_one_and_two_data.csv", index_col=0)
df.head(10)

Mounted at /content/drive


,start,home_team_id,home_team_name,competition_id,season_id,competition_name,away_team_id,away_team_name,home_score,away_score,home_xg,away_xg,features_neutral_venue,features_no_crowds
1713,2017-08-12 16:00:00+00:00,71,AFC Wimbledon,3,1,League One,69,Shrewsbury Town,0,1,0.258691,1.166972,False,False
1716,2017-08-12 16:00:00+00:00,87,Bristol Rovers,3,1,League One,78,Peterborough United,1,4,2.123095,3.151059,False,False
1761,2017-08-19 16:00:00+00:00,81,Fleetwood Town,3,1,League One,71,AFC Wimbledon,2,0,0.217756,1.468496,False,False
1760,2017-08-19 16:00:00+00:00,88,Southend United,3,1,League One,90,Plymouth Argyle,1,1,1.131817,0.598636,False,False
1757,2017-08-19 16:00:00+00:00,91,Scunthorpe United,3,1,League One,80,Oxford United,1,0,1.038351,1.076372,False,False
1756,2017-08-19 16:00:00+00:00,78,Peterborough United,3,1,League One,86,Rotherham United,2,1,1.955395,1.670014,False,False
1753,2017-08-19 16:00:00+00:00,73,Portsmouth,3,1,League One,79,Walsall,1,1,0.710397,1.434023,False,False
1755,2017-08-19 16:00:00+00:00,70,Doncaster Rovers,3,1,League One,82,Blackpool,3,3,1.803022,0.630519,False,False
1751,2017-08-19 16:00:00+00:00,84,Oldham Athletic,3,1,League One,85,Wigan Athletic,0,2,0.705178,1.055413,False,False
1750,2017-08-19 16:00:00+00:00,89,Milton Keynes Dons,3,1,League One,74,Gillingham,1,0,2.034986,0.357925,False,False


##2. Data Preparation

Basic feature engineering:

In [2]:
# Establishing a unique match id
df["match_id"] = df.index.astype(int)

# Season identification
seasons_map = {1:2018, 4:2019, 42:2020, 90:2021, 108:2022, 235:2023, 281:2024, 317:2025, 318:2026}
df['season'] = df['season_id'].map(seasons_map)

# League identification
df['league'] = df['competition_name'].apply(lambda x: 1 if x == 'League One' else 2)

Creating model_df which has one row per team per match (two rows for each match). The future models will predict goals for home and away teams independently and combine these expectations to form a total goal prediction.

In [3]:
model_df = df.copy()
model_df = pd.concat([model_df[['match_id','league', 'season', 'start', 'home_team_name','away_team_name', 'home_team_id', 'away_team_id', 'home_score', 'home_xg', 'features_no_crowds', 'features_neutral_venue']].assign(home=1).rename(
            columns={'home_team_name':'attack', 'away_team_name':'defence', 'home_team_id':'attack_id', 'away_team_id': 'defence_id', 'home_score':'goals', 'home_xg': 'xg'}),
            model_df[['match_id', 'league', 'season', 'start', 'away_team_name', 'home_team_name', 'home_team_id', 'away_team_id', 'away_score', 'away_xg', 'features_no_crowds', 'features_neutral_venue']].assign(home=0).rename(
            columns={'away_team_name':'attack', 'home_team_name':'defence', 'away_team_id':'attack_id', 'home_team_id': 'defence_id','away_score':'goals', 'away_xg': 'xg'})])
model_df.reset_index(inplace=True, drop=True)


Creating variables to track time through seasons:

In [4]:
# Season match number for attacking and defensive teams
model_df.sort_values(["league", "season", "attack_id", "start"], inplace=True)
model_df["att_match_num"] = model_df.groupby(["league", "season", "attack_id"]).cumcount()
model_df.sort_values(["league", "season", "defence_id", "start"], inplace=True)
model_df["def_match_num"] = model_df.groupby(["league", "season", "defence_id"]).cumcount()

# Season match number assigned to each macth
model_df["season_match_number"] = model_df[["att_match_num", "def_match_num"]].max(axis=1) + 1

Creating binary variables to highlight newly promoted and relegated teams:

In [5]:
# Initiating variables
cols = ['att_pro_L1','def_pro_L1','att_rel_L1','def_rel_L1', 'att_pro_L2','def_pro_L2','att_rel_L2','def_rel_L2']
model_df[cols] = False

# Function to implementing binary promotion/relegation variables
def set_att_def_flags(df, team, season, flag):
    df.loc[(df["attack"] == team) & (df["season"] == season), f"att_{flag}"] = True
    df.loc[(df["defence"] == team) & (df["season"] == season), f"def_{flag}"] = True

# Manually inputting promotion/relegation data for 18/17 and 18/19 seasons
for team in ["Blackburn Rovers", "Wigan Athletic", "Rotherham United"]:
    set_att_def_flags(model_df, team, 2018, "rel_L1")
for team in ["Doncaster Rovers", "Blackpool", "Portsmouth", "Plymouth Argyle"]:
    set_att_def_flags(model_df, team, 2018, "pro_L1")
for team in ["Barnsley", "Burton Albion", "Sunderland"]:
    set_att_def_flags(model_df, team, 2019, "rel_L1")
for team in ["Accrington Stanley", "Luton Town", "Coventry City", "Wycombe Wanderers"]:
    set_att_def_flags(model_df, team, 2019, "pro_L1")
for team in ["Bury", "Milton Keynes Dons", "Oldham Athletic ", "Northampton Town"]:
    set_att_def_flags(model_df, team, 2019, "rel_L2")
for team in ["Macclesfield Town", "Tranmere Rovers"]:
    set_att_def_flags(model_df, team, 2019, "pro_L2")

# Logic to identify promoted/relegated teams
for season in sorted(model_df["season"].unique())[2:]:

    # League comopositions for a given and prior season
    L1_teams = model_df.loc[(model_df["league"] == 1) & (model_df["season"] == season), "attack"].unique()
    L1_prev = model_df.loc[(model_df["league"] == 1) & (model_df["season"] == season - 1), "attack"].unique()
    L2_teams = model_df.loc[(model_df["league"] == 2) & (model_df["season"] == season), "attack"].unique()
    L2_prev = model_df.loc[(model_df["league"] == 2) & (model_df["season"] == season - 1), "attack"].unique()

    # Looping through all teams
    for team in model_df["attack"].unique():

        # Relegated from Championship to League One
        if (team in L1_teams) and (team not in L1_prev) and (team not in L2_prev):
            set_att_def_flags(model_df, team, season, "rel_L1")

        # Promoted from League Two to League One
        elif (team in L1_teams) and (team in L2_prev):
            set_att_def_flags(model_df, team, season, "pro_L1")

        # Relegated from League One to League Two
        elif (team in L2_teams) and (team in L1_prev):
            set_att_def_flags(model_df, team, season, "rel_L2")

        # Promoted from National League to League Two
        elif (team in L2_teams) and (team not in L2_prev) and (team not in L1_prev):
            set_att_def_flags(model_df, team, season, "pro_L2")

# Checking that the pipeline has worked correctly
team_season = (model_df[["season", "league", "attack", "att_pro_L1", "att_pro_L2", "att_rel_L2", "att_rel_L1"]].drop_duplicates(subset=["season", "league", "attack"]).copy())
for season in sorted(team_season["season"].unique()):
    season_df = team_season[team_season["season"] == season]
    print(f"{season-1}/{season}")
    print(f"League {1}:")
    print(f"Promoted teams: {sorted(season_df.loc[season_df['att_pro_L1'], 'attack'].unique())}")
    print(f"Relegated teams: {sorted(season_df.loc[season_df['att_rel_L1'], 'attack'].unique())}")
    print(f"League {2}:")
    print(f"Promoted teams: {sorted(season_df.loc[season_df['att_pro_L2'], 'attack'].unique())}")
    print(f"Relegated teams: {sorted(season_df.loc[season_df['att_rel_L2'], 'attack'].unique())}")

2017/2018
League 1:
Promoted teams: ['Blackpool', 'Doncaster Rovers', 'Plymouth Argyle', 'Portsmouth']
Relegated teams: ['Blackburn Rovers', 'Rotherham United', 'Wigan Athletic']
League 2:
Promoted teams: []
Relegated teams: []
2018/2019
League 1:
Promoted teams: ['Accrington Stanley', 'Coventry City', 'Luton Town', 'Wycombe Wanderers']
Relegated teams: ['Barnsley', 'Burton Albion', 'Sunderland']
League 2:
Promoted teams: ['Macclesfield Town', 'Tranmere Rovers']
Relegated teams: ['Bury', 'Milton Keynes Dons', 'Northampton Town', 'Oldham Athletic ']
2019/2020
League 1:
Promoted teams: ['Lincoln City', 'Milton Keynes Dons', 'Tranmere Rovers']
Relegated teams: ['Bolton Wanderers', 'Ipswich Town', 'Rotherham United']
League 2:
Promoted teams: ['Leyton Orient', 'Salford City']
Relegated teams: ['Bradford City', 'Plymouth Argyle', 'Scunthorpe United', 'Walsall']
2020/2021
League 1:
Promoted teams: ['Crewe Alexandra', 'Northampton Town', 'Plymouth Argyle', 'Swindon Town']
Relegated teams: ['C

Creating a variable to measure the fixture congestion disparity between two teams, such that teams with less congested fixture schedules than their opponents will have a positive value and vice versa. This variable considers the number of past and upcoming games in a 10-day window, assuming that fatigue and rotation will harm teams with more congested schedules.

In [6]:
# creating a schedule per team
team_sched = (model_df[["league", "season", "start", "match_id", "attack_id"]].rename(columns={"attack_id": "team_id"})
              .sort_values(["league", "season", "team_id", "start"]).reset_index(drop=True))

# Define lookup
starts_lookup = {key: game["start"].to_numpy(dtype="datetime64[ns]") for key, game in team_sched.groupby(["league", "season", "team_id"], sort=False)}

# Looking at a window of 10 games either side of the current fixture
delta = np.timedelta64(10, "D")

# Initiate new columns
model_df["att_congestion"] = 0
model_df["def_congestion"] = 0


# Calculate congestion for the attacking team
for (league, season, team_id), idx in model_df.groupby(["league", "season", "attack_id"], sort=False).groups.items():
    starts = starts_lookup.get((league, season, team_id))
    t = model_df.loc[idx, "start"].to_numpy(dtype="datetime64[ns]")
    left = np.searchsorted(starts, t - delta, side="left")
    right = np.searchsorted(starts, t + delta, side="right")
    model_df.loc[idx, "att_congestion"] = (right - left).astype(int)

# Calculate congestion for the defending team
for (league, season, team_id), idx in model_df.groupby(["league", "season", "defence_id"], sort=False).groups.items():
    starts = starts_lookup.get((league, season, team_id))
    t = model_df.loc[idx, "start"].to_numpy(dtype="datetime64[ns]")
    left = np.searchsorted(starts, t - delta, side="left")
    right = np.searchsorted(starts, t + delta, side="right")
    model_df.loc[idx, "def_congestion"] = (right - left).astype(int)

# Creating congestion advantage variable as the difference between defending and attacking teams
model_df["congestion_ad"] = (model_df["def_congestion"] - model_df["att_congestion"])


/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: UserWarning: no explicit representation of timezones available for np.datetime64
  result = np.asarray(values, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: UserWarning: no explicit representation of timezones available for np.datetime64
  result = np.asarray(values, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: UserWarning: no explicit representation of timezones available for np.datetime64
  result = np.asarray(values, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: UserWarning: no explicit representation of timezones available for np.datetime64
  result = np.asarray(values, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: UserWarning: no explicit representation of timezones available for np.datetime64
  result = np.asarray(values, dtype=dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/bas

In [7]:
# Inspecting the dataframe
model_df.head()

,match_id,league,season,start,attack,defence,attack_id,defence_id,goals,xg,...,def_pro_L1,att_rel_L1,def_rel_L1,att_pro_L2,def_pro_L2,att_rel_L2,def_rel_L2,att_congestion,def_congestion,congestion_ad
8168,1887,1,2018,2017-09-09 16:00:00+00:00,Blackburn Rovers,Rochdale,75,68,3,3.132633,...,False,True,False,False,False,False,False,3,3,0
8172,1931,1,2018,2017-09-12 20:45:00+00:00,Doncaster Rovers,Rochdale,70,68,1,1.110031,...,False,False,False,False,False,False,False,3,3,0
51,1967,1,2018,2017-09-16 16:00:00+00:00,Milton Keynes Dons,Rochdale,89,68,3,1.316694,...,False,False,False,False,False,False,False,2,4,2
8195,2077,1,2018,2017-09-23 16:00:00+00:00,Gillingham,Rochdale,74,68,0,0.561275,...,False,False,False,False,False,False,False,4,4,0
75,2123,1,2018,2017-09-26 20:45:00+00:00,Blackpool,Rochdale,82,68,0,1.017046,...,False,False,False,False,False,False,False,3,3,0


##3. Export Prepared Dataframe

In [8]:
# Downloading as csv file
model_df.to_csv("/content/drive/My Drive/Goal_Prediction_Model/data/prepared_data.csv", index=False)